# Pretrain BigBird on GENCODE

This notebook uses the `transformers` library to pretrain the BigBird architecture on GENCODE using MLM loss. Adapted from Hugging Face script https://github.com/huggingface/transformers/blob/master/examples/legacy/run_language_modeling.py.

In [1]:
from tokenizers import Tokenizer
from tokenizers.trainers import WordLevelTrainer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import Whitespace

from transformers import PreTrainedTokenizerFast
from transformers import AutoConfig
from transformers import BigBirdForMaskedLM
from transformers import DataCollatorForLanguageModeling
from transformers import LineByLineTextDataset
from transformers import Trainer, TrainingArguments
from transformers import pipeline

import torch
import math

In [2]:
!nvidia-smi

Fri Nov  5 14:07:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           On   | 00000001:00:00.0 Off |                    0 |
| N/A   50C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
torch.cuda.is_available()

True

## Train the tokenizer

In [4]:
paths = ["./data/6mers_train_tiny.txt", "./data/6mers_test_tiny.txt"]

# Initialize a tokenizer
tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
tokenizer.pre_tokenizer = Whitespace()

# Customize training
trainer = WordLevelTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer.train(files=paths, trainer=trainer)

# Convert to PretrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    unk_token="[UNK]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    pad_token="[PAD]",
    mask_token="[MASK]",
)

In [5]:
tokenizer.get_vocab()

{'AAGGCT': 536,
 'GAGCAT': 897,
 'GAGTTT': 351,
 'ACCCCA': 610,
 'GAATAC': 715,
 'CTCCAT': 963,
 'ATATAT': 1035,
 'TGCACT': 1258,
 'AGCCTT': 1396,
 'GTCTAC': 895,
 'GGAGGT': 1046,
 'TCAAAC': 1435,
 'TTAGAT': 858,
 'GGAGCT': 779,
 'TTACTT': 1482,
 'AGGGAA': 500,
 'CTGTTG': 939,
 'GCAGCC': 1401,
 'TCACTA': 840,
 'CATCAT': 1427,
 'TGGGGG': 1021,
 'AGTCCT': 1262,
 'GATACA': 170,
 'GATAAC': 630,
 'TTTGGC': 889,
 'AAGCAT': 1449,
 'TGTAGC': 486,
 'GTATAG': 1279,
 'CAACAT': 159,
 'CTTAGT': 593,
 'TGACTG': 1329,
 'CTCACG': 1515,
 'AAGCCC': 1292,
 'CTTGCT': 123,
 'GATGGA': 84,
 'AGCTAA': 1067,
 'GCTTTT': 53,
 'CACAGT': 477,
 'TAGCTG': 596,
 'CCAATA': 372,
 'GTGTGC': 629,
 'ACGAAT': 855,
 'AAGGGT': 1065,
 'AATACC': 933,
 'AGTGTT': 312,
 'AGCCAC': 1070,
 'ACACCA': 1177,
 'ATTAGG': 997,
 'TAGTGT': 424,
 'GTTTGG': 1136,
 'GGTACA': 323,
 'CAGGTG': 383,
 'CTGTAC': 304,
 'ATATCT': 1205,
 'TAGAAT': 1425,
 'TAAAAT': 38,
 'AGTGGT': 1521,
 'CTAACA': 555,
 'GAGTGA': 695,
 'CCCGGG': 614,
 'CTCTCA': 797,
 'CC

## Train language model from scratch

In [6]:
model_checkpoint = "google/bigbird-roberta-base"
config = AutoConfig.from_pretrained(model_checkpoint)
model = BigBirdForMaskedLM(config=config)

In [7]:
model.num_parameters()

128111286

In [8]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [9]:
train_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=paths[0],
    block_size=128,
)

eval_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=paths[1],
    block_size=128,
)

/anaconda/envs/api_env_dev/lib/python3.7/site-packages/transformers/data/datasets/language_modeling.py:124: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [10]:
output_model_dir = "./output_model"
output_tokenizer_dir = "./output_tokenizer"

training_args = TrainingArguments(
    output_dir=output_model_dir,
    overwrite_output_dir=True,
    num_train_epochs=50,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

In [11]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
***** Running training *****
  Num examples = 4
  Num Epochs = 50
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 50
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Attention type 'block_sparse' is not possible if sequence_length: 128 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




Attempted to log scalar metric train_runtime:
17.1734
Attempted to log scalar metric train_samples_per_second:
11.646
Attempted to log scalar metric train_steps_per_second:
2.911
Attempted to log scalar metric total_flos:
13254002380800.0
Attempted to log scalar metric train_loss:
9.860770263671874
Attempted to log scalar metric epoch:
50.0
CPU times: user 15.7 s, sys: 1.5 s, total: 17.2 s
Wall time: 17.2 s


TrainOutput(global_step=50, training_loss=9.860770263671874, metrics={'train_runtime': 17.1734, 'train_samples_per_second': 11.646, 'train_steps_per_second': 2.911, 'total_flos': 13254002380800.0, 'train_loss': 9.860770263671874, 'epoch': 50.0})

In [12]:
trainer.save_model(output_model_dir)
tokenizer.save_pretrained(output_tokenizer_dir)

Saving model checkpoint to ./output_model
Configuration saved in ./output_model/config.json
Model weights saved in ./output_model/pytorch_model.bin
tokenizer config file saved in ./output_tokenizer/tokenizer_config.json
Special tokens file saved in ./output_tokenizer/special_tokens_map.json


('./output_tokenizer/tokenizer_config.json',
 './output_tokenizer/special_tokens_map.json',
 './output_tokenizer/tokenizer.json')

# Validate training

In [13]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 4
  Batch size = 8


Attempted to log scalar metric eval_loss:
10.80819320678711
Attempted to log scalar metric eval_runtime:
0.0959
Attempted to log scalar metric eval_samples_per_second:
41.72
Attempted to log scalar metric eval_steps_per_second:
10.43
Attempted to log scalar metric epoch:
50.0
Perplexity: 49424.09


In [14]:
fill_mask = pipeline(
    "fill-mask",
    model=output_model_dir,
    tokenizer=output_tokenizer_dir,
)

loading configuration file ./output_model/config.json
Model config BigBirdConfig {
  "architectures": [
    "BigBirdForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "attention_type": "block_sparse",
  "block_size": 64,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu_new",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 4096,
  "model_type": "big_bird",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_random_blocks": 3,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "rescale_embeddings": false,
  "sep_token_id": 66,
  "torch_dtype": "float32",
  "transformers_version": "4.9.0",
  "type_vocab_size": 2,
  "use_bias": true,
  "use_cache": true,
  "vocab_size": 50358
}

loading configuration file ./output_model/config.json
Model config BigBirdConfig {
  "architectures": [
    "BigBird

In [15]:
fill_mask("TTAGTT TAGTTT [MASK]")

Attention type 'block_sparse' is not possible if sequence_length: 3 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3.Changing attention type to 'original_full'...


[{'sequence': 'TTAGTT TAGTTT GGTGAT',
  'score': 0.0002999096759594977,
  'token': 239,
  'token_str': 'GGTGAT'},
 {'sequence': 'TTAGTT TAGTTT GCTTAG',
  'score': 0.00028870513779111207,
  'token': 404,
  'token_str': 'GCTTAG'},
 {'sequence': 'TTAGTT TAGTTT AGGAAC',
  'score': 0.00026762159541249275,
  'token': 1270,
  'token_str': 'AGGAAC'},
 {'sequence': 'TTAGTT TAGTTT GCAATC',
  'score': 0.0002277830644743517,
  'token': 517,
  'token_str': 'GCAATC'},
 {'sequence': 'TTAGTT TAGTTT ATCACA',
  'score': 0.00020295516878832132,
  'token': 1346,
  'token_str': 'ATCACA'}]